In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import nltk
from nltk import tokenize
from gensim.parsing.preprocessing import remove_stopwords
import numpy as np
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# upload the csv file to colab and make sure it is fully downloaded before progressing
fname = "renewables_f1000_11_07_2023 with db description.csv"

In [ ]:
data = pd.read_csv(fname, header=0,dtype={'Paragraph':'string'}).reset_index()

In [ ]:
data.head()

,index,db,Title,DOI,Authors,Full_Text_URL,Paragraph,Question,Answer,Column1
0,0,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,Figure 1. Trend in allocation of total NIH bu...,What percentage of the Total NIH Budget is all...,The percentage of the Total NIH Budget allocat...,NaN
1,1,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,Figure 1. Trend in allocation of total NIH bu...,How much of the Total NIH Budget was allocated...,The percentage of the Total NIH Budget allocat...,NaN
2,2,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,A few data snapshots bear particular comment....,What is the full post-doubling era?,The full post-doubling era is the period betwe...,NaN
3,3,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,A few data snapshots bear particular comment....,How did the NIH budget increase during the pos...,The NIH budget increased by $2.8 billion durin...,NaN
4,4,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,"Analogously, the evolving NIH philosophy appe...",What is the new NIH philosophy?,The new NIH philosophy appears to tilt more to...,NaN


In [ ]:
# chosen_columns = ['Energy & climate', 'Food']

In [ ]:
# data = data.loc[data['db'].isin(chosen_columns)]

In [ ]:
data["subpara"] = ""

In [ ]:
from collections import Counter
Counter(data['db'])

Counter({'biotech': 446,
         'Energy & climate': 784,
         'Finance': 74,
         'Food': 210,
         'Data & Finance ': 28,
         'Biotech': 1344,
         'International ': 468})

In [ ]:
def getTopSents(top_sent_indices, cosine_similarities, sent_set):
  pointer = 0
  while pointer < len(top_sent_indices):
    if pointer > 0 and cosine_similarities[top_sent_indices[pointer]] < 0.5:
      break
    sent_set.add(top_sent_indices[pointer])
    pointer += 1
  return sent_set

In [ ]:
def method1():
  tfidf_vectorizer = TfidfVectorizer()
  for index, row in data.iterrows():
    paragraph = row['Paragraph']
    sentences = tokenize.sent_tokenize(paragraph)
    sent_vecs = tfidf_vectorizer.fit_transform([remove_stopwords(text) for text in sentences])
    try:
      q_vec = tfidf_vectorizer.transform([remove_stopwords(row['Question'])])
      a_vec = tfidf_vectorizer.transform([remove_stopwords(row['Answer'])])
    except:
      print("Skipped index:", index)
      continue
    q_cosine_similarities = linear_kernel(sent_vecs, q_vec).flatten()
    a_cosine_similarities = linear_kernel(sent_vecs, a_vec).flatten()
    # print(top_sent_indices)
    q_top_sent_indices = q_cosine_similarities.argsort()[::-1]
    a_top_sent_indices = a_cosine_similarities.argsort()[::-1]
    # chosen_sentences = set()
    aPointer = 0
    qPointer = 0
    chosen_sentences_indices = getTopSents(q_top_sent_indices, q_cosine_similarities, set())
    chosen_sentences_indices = getTopSents(a_top_sent_indices, a_cosine_similarities, chosen_sentences_indices)

    chosen_sentences = " ".join(sentences[i] for i in sorted(chosen_sentences_indices))

    data.at[index,'subpara'] = chosen_sentences
  return data

In [ ]:
def method2():
  for index, row in data.iterrows():
    paragraph = row['Paragraph']
    sentences = tokenize.sent_tokenize(paragraph)
    sent_sets = [set(sent.split(" ")) for sent in sentences]
    try:
      QA_set = set(remove_stopwords(row['Question']).split(" ")).union(set(remove_stopwords(row['Answer']).split(" ")))
    except:
      print("Skipped index:", index)
      continue
    sent_score = [0]*len(sentences)
    for word in QA_set:
      for i in range(len(sentences)):
        if word in sent_sets[i]:
          sent_score[i] += 1
    sent_indices = np.array(sent_score).argsort()[::-1]
    chosen_sentence_indices = []
    for si in sent_indices:
      if len(sent_sets[si].intersection(QA_set)) > 1:
        chosen_sentence_indices.append(si)
        QA_set = QA_set - sent_sets[si]

    chosen_sentences = " ".join(sentences[i] for i in sorted(chosen_sentence_indices))
    data.at[index,'subpara'] = chosen_sentences

  return data



In [ ]:
# data = method1()
data2 = method2()

Skipped index: 176
Skipped index: 633
Skipped index: 827
Skipped index: 1646
Skipped index: 2027
Skipped index: 2028
Skipped index: 2513
Skipped index: 2672
Skipped index: 2685
Skipped index: 2770
Skipped index: 3299


In [ ]:
data2.head()

,index,db,Title,DOI,Authors,Full_Text_URL,Paragraph,Question,Answer,Column1,subpara
0,0,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,Figure 1. Trend in allocation of total NIH bu...,What percentage of the Total NIH Budget is all...,The percentage of the Total NIH Budget allocat...,NaN,Trend in allocation of total NIH budget to NIH...
1,1,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,Figure 1. Trend in allocation of total NIH bu...,How much of the Total NIH Budget was allocated...,The percentage of the Total NIH Budget allocat...,NaN,Total NIH spending on R01-equivalent grants (i...
2,2,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,A few data snapshots bear particular comment....,What is the full post-doubling era?,The full post-doubling era is the period betwe...,NaN,The full post-doubling era (FY2003–2008) does ...
3,3,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,A few data snapshots bear particular comment....,How did the NIH budget increase during the pos...,The NIH budget increased by $2.8 billion durin...,NaN,The full post-doubling era (FY2003–2008) does ...
4,4,biotech,"NIH Portfolio Allocation, Lemmings, and the Si...",10.12688/f1000research.1-5.v1,"[{'Mark', 'Boothby'}]",https://f1000research.com/extapi/article/xml?d...,"Analogously, the evolving NIH philosophy appe...",What is the new NIH philosophy?,The new NIH philosophy appears to tilt more to...,NaN,"Analogously, the evolving NIH philosophy appe..."


In [ ]:
data.to_csv(f"processed1_{fname}")

In [ ]:
data2.to_csv(f"processed3_{fname}")